In [5]:
import pandas as pd

In [6]:
npr = pd.read_csv('npr.csv')

In [7]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [8]:
#to see article, index row
#print(npr['Article'][1])

#to see how many articles
len(npr)

11992

# before LDA, need to do some preprocessing

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [10]:
#no train test split, since unsupervised.  So fittransform to all data


In [11]:
dtm = tfidf.fit_transform(npr['Article'])

In [12]:
dtm #should be a sparse matrix

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [13]:
#get 54,777 terms in teh 11,992 articles

In [15]:
from sklearn.decomposition import NMF

In [16]:
nmf_model = NMF(n_components=7, random_state=42)

In [17]:
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [18]:
#NMF is faster the LDA

In [18]:
# 3 steps left
# Grab vocabulary of words

# Grab the topics

# Grab the highest probability words per topic

In [19]:
# Grab vocab of words
len(tfidf.get_feature_names())

54777

In [21]:
type(tfidf.get_feature_names())

list

In [22]:
# getting words at index locations
tfidf.get_feature_names()[41000]

'reproductive'

In [23]:
for index, topic in enumerate(nmf_model.components_):
    print(f"The top 15 words for Topic #{index}")
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')
    print('\n')
#with NMF, deal with words that have the highest COEFFICIENTS (not probabilities, like LDA)

The top 15 words for Topic #0
['new', 'research', 'like', 'patients', 'health', 'disease', 'percent', 'women', 'virus', 'study', 'water', 'food', 'people', 'zika', 'says']




The top 15 words for Topic #1
['gop', 'pence', 'presidential', 'russia', 'administration', 'election', 'republican', 'obama', 'white', 'house', 'donald', 'campaign', 'said', 'president', 'trump']




The top 15 words for Topic #2
['senate', 'house', 'people', 'act', 'law', 'tax', 'plan', 'republicans', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']




The top 15 words for Topic #3
['officers', 'syria', 'security', 'department', 'law', 'isis', 'russia', 'government', 'state', 'attack', 'president', 'reports', 'court', 'said', 'police']




The top 15 words for Topic #4
['primary', 'cruz', 'election', 'democrats', 'percent', 'party', 'delegates', 'vote', 'state', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']




The top 15 words for Topic #5
['love', 've', 

# now last thing, attached topic numbers to original articles

In [24]:
topic_results = nmf_model.transform(dtm)

In [25]:
topic_results.shape #number of articles with probabilities of each topic

(11992, 7)

In [26]:
topic_results[1].round(2)

array([0.01, 0.13, 0.  , 0.02, 0.  , 0.  , 0.  ])

In [27]:
#most probably topic 1 - the one about trump (see article up top)

In [28]:
#now connect back to original dataframe
#use ARGMAX to return index position of highest probability

In [29]:
npr['Topic'] = topic_results.argmax(axis = 1)

In [30]:
npr.head(15)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,5
6,With a who has publicly supported the debunk...,0
7,"I was standing by the airport exit, debating w...",0
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",5


# create dictionary of topics

In [31]:
my_topic_dict = {0:'Health', 1:'Politics',2:'Legislation', 3:'Foreign Policy', 4:'Election', 5:'Music', 6: 'Education'}
npr['Topic Label'] = npr['Topic'].map(my_topic_dict)


In [33]:
npr.head(15)

,Article,Topic,Topic Label
0,"In the Washington of 2016, even when the polic...",1,Politics
1,Donald Trump has used Twitter — his prefe...,1,Politics
2,Donald Trump is unabashedly praising Russian...,1,Politics
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3,Foreign Policy
4,"From photography, illustration and video, to d...",6,Education
5,I did not want to join yoga class. I hated tho...,5,Music
6,With a who has publicly supported the debunk...,0,Health
7,"I was standing by the airport exit, debating w...",0,Health
8,"If movies were trying to be more realistic, pe...",0,Health
9,"Eighteen years ago, on New Year’s Eve, David F...",5,Music
